
# Beyond CFT: Attacks on the Network and Convergence 


So far we showed how gossip can withstand network imperfection. But what if the attacker deliberately splits and attacks the network?  


Beyond regular crashes, peer can behave in various ways violating the protocol: hide transactions, send bogus data, create Sybil entities etc.
The goal of a blockchain system is to withstand against a powerful adversary. 

To ensure that message will be seen by the peer, once the peer is back online it must fetch the data from the neighboring peers. But what if the neighboring nodes are malicious and will censor certain transactions?   

In the next notebooks we will cover techniques that help to detect/prevent malicious behaviour.



# Malicious gossip agent 

One of the goal of a blockchain system is to record transaction in a 'hard-to-tamper' way.

How can you achieve that in P2P settings?  
It is common in databases and blockchains to use cryptography to verify the integrities of the transactions.

Let's first create a malicious agent that will change the data of received transactions to split the network.  


In [52]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper


## Define malicious agents 
Let's first add malicious nodes randomly: 

In [53]:
# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.3 # 30 % of malicious nodes


def assign_malicious_peers(topology, mal_frac):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(frac_malicious_nodes * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)

## Define malicious services 

We will inherit a malicious gossip service that will relay the gossip message to one half of the network and the other half a tempered message (with different data). 



In [54]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)

##  Add malicious type and services 

We deliberately keep malicious nodes uncrashable. 


In [55]:
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = p2p.GossipService



peer_services['malicious'] = p2p.PeerType(peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )

## Run simulation 

Let's see how malicious agents together with crashing nodes affect the message dissemination. 

In [56]:
serv_impl

{'BaseConnectionManager': p2psimpy.services.connection_manager.BaseConnectionManager,
 'MessageProducer': p2psimpy.services.message_producer.MessageProducer,
 'RandomDowntime': p2psimpy.services.disruption.RandomDowntime,
 'RangedPullGossipService': p2psimpy.services.gossip.GossipService}

In [39]:
from p2psimpy.messages import GossipMessage

In [40]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(3_200)

# Analyze the storage data




## Message data

Let's see how this fraction of malicious nodes affected the network. 
We compare the received message with the original message, we report `True` if the message wasn't tampered `False` and otherwise. 



In [41]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

,3,4,5,7,8,9,11,12,13,14,15
1,False,False,False,False,False,False,False,False,False,False,True
2,False,True,False,True,True,True,True,False,False,False,True
3,True,True,True,True,False,True,True,True,True,False,False
4,False,True,True,True,True,False,False,True,True,False,True
5,False,False,False,False,False,True,False,False,False,False,True
6,True,False,False,True,False,False,False,False,True,False,False
7,False,False,True,False,False,False,True,False,True,False,True
8,True,False,False,True,False,True,False,True,True,False,False
9,False,True,False,False,True,False,False,True,False,False,False
10,False,False,True,True,NaN,NaN,NaN,False,False,True,False


In [42]:
df[df==False].count()

3     7
4     6
5     6
7     4
8     7
9     6
11    6
12    6
13    5
14    9
15    6
dtype: int64

Malicious nodes managed to trick some peers into accepting wrong data! As peers will write 'first-seen' value, adversary once having advantage over the network can perfectly split the network. 

How to deal with this?

# Signing messages

First of all, messages themselves must be verified on their **integrity** and **authenticity**. 
[Digital signatures](https://en.wikipedia.org/wiki/Digital_signature) are perfect match for this and hence all blockchain systems use them. 

We will modify the code to simulate the signed messages. We will not use an actual crytpographic protocol since we care about only two things for our simulation: 
- It takes time to verify and sign messages. 
- Peers should store and forward only valid messages. 



## Simulating digital signatures 

We will show an example by building a crypto validator for 456 bits [EdDSA (Ed448)](https://en.wikipedia.org/wiki/EdDSA) (one of the most popular digital signatures in the wild).
On a regular laptop it takes usually less than 1 millisecond to verify a signature. Let's take the near worse case.  


This is not real. 

We will integrate a verification task into the message itself. 
Peer before triggering other services will first run the `pre_task`.  

Since the message is first created by MessageProducer we will add a task in the configuration.



In [43]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist


conf = peer_services['client'].service_map['MessageProducer']


def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        print("FALSE")
        return False
    return True
    print("TRUE")


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig

In [44]:
# Run the simulation with a modificed message producer 

sim2 = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(3_200)


FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE


In [45]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df

,3,4,5,7,8,9,11,12,13,14,15
1,True,True,True,True,NaN,True,True,True,True,NaN,NaN
2,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,NaN,NaN,True,True
4,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,NaN,NaN,True,NaN
7,True,True,True,True,True,NaN,True,True,NaN,True,NaN
8,NaN,NaN,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN
9,True,True,True,True,True,True,NaN,True,NaN,True,True
10,NaN,True,True,True,True,True,True,True,True,True,True


In [46]:
sim2.peers[12].storage['msg_data'].txs

{'17_1': GossipMessage:QWYPNAQWXTVHGBUZAJBK,
 '16_1': GossipMessage:EZOQTONFGRQLXWXRHGOA,
 '16_2': GossipMessage:OFAJBTQPXVPERBTTCAVS,
 '17_2': GossipMessage:WONUYISTSABOSEXYHFTU,
 '16_4': GossipMessage:JCFYNBABXVDGOHTDEOHD,
 '17_4': GossipMessage:DYQOMCWENWVAXUBFCTHC,
 '16_5': GossipMessage:DIWTUMVWQJXGFIVCNJEV,
 '17_7': GossipMessage:VRUBFBZJREKORRGSZSFD,
 '16_7': GossipMessage:KYGCYLCTSOQVXUJLNUUQ,
 '16_9': GossipMessage:HAGOOEROFPLZWMFYBURW,
 '17_10': GossipMessage:CMTHWHJANRAGAXWMZOPS,
 '16_10': GossipMessage:MTZWUHCXJXOYVINSPLVT,
 '16_11': GossipMessage:HAENAVGQRLKBCIWSEXZX,
 '17_11': GossipMessage:LXZFTCCGSTIGBWXISQRP}

Now malicious nodes cannot change the message. They need to explore other attack strategies!
The malicious nodes can delay messages, hide them, freeriding in a gossip (only listening). 
Together with network attack this can create a dangerous combination. 

Let us consider a case where an honest node is surrounded by malicious nodes (all network connections are with malicious nodes) that will hide certain transactions. As a result, peer will not receive crucial transactions that might affect it's decision making process. This attack is also called **Eclipse attack**.   

In reality, almost nothing stops one malicious node from running multiple instances and poison the whole network. This attack is called **Sybil Attack**. 




### Exercises


- Explore the limits of the gossip protocol. What is the maximum number of malicious nodes a protocol can tolerate? 
- Try to eclipse attack some peer, make sure he doesn't get any message, or one specific message (censor)? 



In [57]:
#To check how many malicious nodes this protocol can handle, I increase their percentage and then apply the technique of digital signature in order to see 
#if I still can receive the correct messages at the end. This can be checked by the proof matrix. 

# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper

# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.6 


def assign_malicious_peers(topology, mal_frac):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(frac_malicious_nodes * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)



In [58]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)
            
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = p2p.GossipService



peer_services['malicious'] = p2p.PeerType(peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )


serv_impl

{'BaseConnectionManager': p2psimpy.services.connection_manager.BaseConnectionManager,
 'MessageProducer': p2psimpy.services.message_producer.MessageProducer,
 'RandomDowntime': p2psimpy.services.disruption.RandomDowntime,
 'RangedPullGossipService': p2psimpy.services.gossip.GossipService}

In [59]:
from p2psimpy.messages import GossipMessage

# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(3_200)

import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df


,2,4,6,10,11,14
1,True,False,False,True,False,False
2,True,True,True,False,True,False
3,False,False,False,False,False,False
4,False,False,False,False,False,True
5,False,False,False,NaN,False,True
6,False,NaN,False,False,False,False
7,False,False,False,False,True,False
8,False,True,NaN,NaN,True,True
9,NaN,False,False,False,True,True
10,False,False,False,False,False,False


In [61]:
#as seen above, with 60% of nodes being malicious, the messageID: 10 became censored as it was not received by any node (i.e., False messages in the proof-matrix)

df[df==False].count()

2     7
4     8
6     8
10    8
11    7
14    7
dtype: int64

In [67]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist


conf = peer_services['client'].service_map['MessageProducer']


def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        #print("FALSE")
        return False
    return True
    #print("TRUE")


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig


# Run the simulation with a modificed message producer 

sim2 = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(3_200)



In [68]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df


,2,4,6,10,11,14
1,True,True,NaN,True,True,True
2,True,True,NaN,NaN,NaN,NaN
3,NaN,True,True,True,True,True
4,NaN,True,NaN,True,True,True
5,True,True,NaN,True,True,True
6,True,NaN,True,True,True,NaN
7,NaN,True,NaN,NaN,NaN,NaN
8,True,NaN,NaN,NaN,True,NaN
9,True,True,NaN,NaN,NaN,NaN
10,True,True,NaN,True,NaN,True


In [69]:
sim2.peers[12].storage['msg_data'].txs

{'16_2': 'PTFOUHQZQXUJYGAMSSWM',
 '17_3': 'EPOIBDPYMUHTPAFKDTNU',
 '17_6': 'BDTQDHMZSEMUPFNENGEB',
 '16_7': 'RLCONHYGUJRDJIOJCGBY',
 '16_9': 'ZURRYSFRCBDRROVYPUYB',
 '16_10': 'AUZRIQKSGUNNEDRDGLUJ'}

In [71]:
#however, as seen above, signing messages helped to turn the situation, because no node accepted "False" data. 
#The price that was paid for that was more NaN situations as many messages were not received at all.
#However, better not received than received as False, because a message can always be retransmitted but a falsely received message can reduce the network performance. 

In [178]:
#if I increase the percentage of malicious nodes even more, we can also see cases of eclipsing

# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper

# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.80 


def assign_malicious_peers(topology, mal_frac):
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(frac_malicious_nodes * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)



In [179]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)
            
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = p2p.GossipService



peer_services['malicious'] = p2p.PeerType(peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )


serv_impl

{'BaseConnectionManager': p2psimpy.services.connection_manager.BaseConnectionManager,
 'MessageProducer': p2psimpy.services.message_producer.MessageProducer,
 'RandomDowntime': p2psimpy.services.disruption.RandomDowntime,
 'RangedPullGossipService': p2psimpy.services.gossip.GossipService}

In [180]:
from p2psimpy.messages import GossipMessage

# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(3_200)

import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df


,12,14,7
1,True,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,NaN,False,True
10,False,False,False


In [181]:
#as seen above, with 80% of nodes being malicious, the nodeID: 14 suffered from eclipse-attack as all messages it received were "False". 
#Notice that because of the high robustness of the network I chose (as it is invoked from the previous exercises), the average high degree of 
#each node make it hard for someone to be "surrounded" only by malicious peers

df[df==False].count()

12     9
14    11
7     10
dtype: int64

In [182]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist


conf = peer_services['client'].service_map['MessageProducer']


def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        #print("FALSE")
        return False
    return True
    #print("TRUE")


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig


# Run the simulation with a modificed message producer 

sim2 = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(3_200)



In [183]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df


,12,14,7
1,True,True,NaN
2,NaN,True,True
3,NaN,True,True
4,True,True,NaN
8,True,True,NaN
10,NaN,NaN,True
11,NaN,NaN,True


In [184]:
sim2.peers[12].storage['msg_data'].txs

{'17_1': GossipMessage:IHDKZSZOXUERWNTFHVEO,
 '16_4': GossipMessage:ORKCSBINCJZEXHQMMBML,
 '17_8': GossipMessage:ZIYFNQCVLHKLVTWLNPTX}

In [185]:
#Signing messages still helped as seen above, certain messages to be recovered.

In [193]:
#Although I guess it depends on the type of network-graph, the above exercise gave me the impression that it is easier to censor a message than to eclipse a peer. 